In [1]:
# Read RData
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json
from os import listdir, getcwd
from os.path import isfile, join
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
import numpy as np
import re

# Read all files in a directory
mypath = getcwd()+"/data/tweet_youtube"
onlyfiles = [ join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
tweets_time = {}
for f in onlyfiles:
    ro.r['load'](f,ro.globalenv)

shares = ro.globalenv[str('data')].rx2("numShare")
views = ro.globalenv[str('data')].rx2("dailyViewcount")

In [13]:
def build_index2popularity_scale(index2views):
    sorted_index2views = sorted(index2views, key=lambda tup: tup[1]) 
    split_index2views = np.array_split(sorted_index2views,40)
    # Build a hashmap : the index of a video -> popularity_scale
    index2ps = {}
    for scale in range(40):
        for element in split_index2views[scale]:
            index2ps[element[0]] = scale
    
    return index2ps

f = open("results3.txt","r")
lines = f.readlines()
lines = [ l.strip() for l in lines]
lines = [ re.split("[\(,\) ]+", l)[1:3] for l in lines]
pred_index2p = [ (int(l[0]), float(l[1])) for l in lines]

viewcounts_120 = [(ele[0], sum(views[int(ele[0])][:120])) for ele in pred_index2p if ele[0]<7300]

pred_index2ps = build_index2popularity_scale(pred_index2p)
real_index2ps = build_index2popularity_scale(viewcounts_120)

In [14]:
L = len(real_index2ps)
sum_ps_error = 0
for index in real_index2ps:
    ps1 = real_index2ps[index]
    ps2 = pred_index2ps[index]
    sum_ps_error = sum_ps_error+abs(ps1-ps2)

mean_abs_percentile_error = sum_ps_error/L/39
print(mean_abs_percentile_error)

0.0200383121562


In [15]:
indexes = [ ele[0]  for ele in pred_index2p]
s = set()
for index in indexes:
    if indexes.count(index)>1:
        s.add(index)

In [12]:
print(s)

set([999, 1465, 1466, 1467, 1468, 1469, 1470])
